In [1]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 


import bk.load
import bk.signal
import bk.plot


import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [2]:
# bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
# bk.load.session = 'Rat08-20130713'


# session = 'Rat08-20130713'
# neurons,shanks = bk.load.loadSpikeData(bk.load.path)

# run = scipy.io.loadmat(bk.load.path + 'runintervals.mat')
# run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

In [143]:
path = bk.load.sessions()['Path'][5]
bk.load.current_session(path)

neurons,shanks = bk.load.loadSpikeData(bk.load.path)
pos = bk.load.pos()
states = bk.load.states()

run = scipy.io.loadmat('runintervals.mat')
run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

trackrun = scipy.io.loadmat(bk.load.session+'-TrackRunTimes.mat')
trackrun = nts.IntervalSet(trackrun['trackruntimes'][:,0],trackrun['trackruntimes'][:,1],time_units='s')

pos_run = pos.restrict(run)#.restrict(trackrun)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-NeuronsShanks.npy


In [144]:
def computeRateMap(neuron,pos,nxbins=10,nybins=10):
    
    """
        neuron : as Tsd from neuroseries
        pos : as Tsd from neuroseries
        
        output : 
        rate_map : Ratemap in Hz 
        spike_count : number of spikes per spatial bin
        occupancy : number of pos in spatial bin
    """
    
    
    dt = scipy.stats.mode(np.diff(pos.as_units('s').index))[0][0]
    
    pos_spike = pos.realign(neuron)
    
    xbins = np.linspace(pos.x.min(),pos.x.max()+0.01,nxbins)
    ybins = np.linspace(pos.y.min(),pos.y.max()+0.01,nybins)
    
    spike_count,_,_ = np.histogram2d(pos_spike.y,pos_spike.x,[ybins,xbins])
    occupancy,_,_ = np.histogram2d(pos.y,pos.x,[ybins,xbins])
    
    rate_map = spike_count*dt/(occupancy+1)
    rate_map = pd.DataFrame(index = ybins[0:-1]+np.diff(ybins)/2,columns=xbins[0:-1]+np.diff(xbins)/2, data = rate_map)
    
#     plt.figure()
#     plt.subplot(211)

#     plt.plot(pos['x'], pos['y'], alpha = 0.5)
#     plt.scatter(pos_spike['x'], pos_spike['y'], color = 'red', alpha = 0.5, edgecolor = None)
#     for i in xbins: plt.axvline(i)
#     for i in ybins: plt.axhline(i)
#     plt.xlim(xbins[0], xbins[-1])
#     plt.ylim(ybins[0], ybins[-1])
#     plt.subplot(212)
#     plt.imshow(rate_map, origin = 'lower', aspect= 'auto')
# #     plt.clim([0,0.1])
    
    
    return rate_map,spike_count,occupancy

def spatialInformationScore(spike_count,occupancy):
    
    """
        Initial code by KevinAllen
        Adapted by BK 2/11/20
        
        rate_map : number of spike per spatial bin
        occupancy : number of position per spatial bin
    """
    
    
    spike_count = spike_count[occupancy>0]
    occupancy = occupancy[occupancy>0]
    
    rate_map = spike_count/occupancy
    
    mRate = spike_count.sum()/occupancy.sum()
    prob = occupancy/occupancy.sum()
    
    index = rate_map>0
    
    info = np.sum(prob[index] * (rate_map[index]/mRate) * np.log2(rate_map[index]/mRate))
    return info


In [173]:
info = []
for n,s in zip(neurons,shanks):
    _,spike_count,occupancy = computeRateMap(n,pos_run,25,15)
    info.append(spatialInformationScore(spike_count,occupancy))
#     plt.suptitle(str(s) + str(t))
MRL = theta['stats']['hpc'][0][0]['r'][0][0].squeeze()

In [174]:
info = np.array(info).squeeze()
MRL = np.array(MRL)

In [172]:
MRL.squeeze().shape

(252,)

In [184]:
data = {'MRL':MRL,'info':info,'shanks':shanks[:,0]}
data = pd.DataFrame(data)

In [186]:
data['region'] = 'empty'
data['region'][data['shanks']<5] = 'Hpc'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [188]:
sns.scatterplot('info','MRL',data = data,hue = 'region')

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='info', ylabel='MRL'>

In [68]:
a = computeRateMap(neurons[5],pos_run,25,15)
plt.clim([0,0.1])
plt.show()

In [129]:
n

TypeError: __init__() got multiple values for keyword argument 'index'

In [109]:
ypos.max()

110.94

In [105]:
b = 
b

39.06249999870326

In [126]:
# for i in run.as_units('s').iloc():
#     print(i.end)
    
    
ypos.as_units('s')[206:207]

Time (s)
206.0032     41.7100
206.0288     41.7100
206.0544     41.7100
206.0800     41.7100
206.1056     41.7100
206.1312     41.7100
206.1568     41.7100
206.1824     41.7100
206.2080     41.7100
206.2336     41.7100
206.3616    108.3600
206.3872    108.3600
206.4128    108.3600
206.4384    108.3600
206.4640    108.3600
206.4896    108.3600
206.5152    108.3600
206.5408    108.3600
206.5664    108.3600
206.5920    108.3600
206.6176    108.3600
206.6432    108.3600
206.6688    108.3600
206.6944    108.3600
206.7200    108.3600
206.7456    108.3600
206.7712    108.3600
206.7968    108.3600
206.8224    108.3600
206.8480    108.3600
206.8736    108.3600
206.8992    108.3600
206.9248    108.3600
206.9504    108.2740
206.9760    108.1364
Name: y, dtype: float64

In [73]:
theta = scipy.io.loadmat('Billel/ThetaMod/ThetaMod.mat')

0.034682411137910846